# Usage Guide -

https://www1.nyc.gov/assets/tlc/downloads/pdf/trip_record_user_guide.pdf

2018 onwards uses zones

In [4]:
import pandas as pd
import numpy as np

In [5]:
jan_yellow = pd.read_csv('../raw_data/yellow_tripdata_2020-01.csv', encoding='utf8', engine='python')

So, starting out with reading csv file for a single month in the year -

1. Drop all rows, for these columns:
    VendorID
    store_and_fwd_flag
    extra
    mta_tax
    tolls_amount
    improvement_surcharge
    congestion_surcharge
    
2. Drop some rows, based on column values:
    RatecodeID = 99
    payment_type = 5
    
3. Convert obj type in dropoff / pickup to datetime

In [3]:
# Function to preprocess & filter data read in from a csv file

def preprocess(df):
    full_drop_labels = ['VendorID', 'store_and_fwd_flag', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge']
    df.drop(full_drop_labels, axis = 1, inplace = True)
    df = df.loc[df['payment_type'] != 5]
    df = df.loc[df['RatecodeID'] != 99]
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
    df.dropna(subset = ['RatecodeID', 'payment_type'], inplace = True)
    df = df.loc[(df['total_amount'] > 0) & (df['trip_distance'] > 0) & (df['total_amount'] < 500) & (df['passenger_count'] > 0)]
    
    return df

## Now, perform this elementary preprocessing on all month files

In [6]:
jan_yellow = pd.read_csv('../raw_data/yellow_tripdata_2020-01.csv', encoding='utf8', engine='python')
feb_yellow = pd.read_csv('../raw_data/yellow_tripdata_2020-02.csv', encoding='utf8', engine='python')
mar_yellow = pd.read_csv('../raw_data/yellow_tripdata_2020-03.csv', encoding='utf8', engine='python')
apr_yellow = pd.read_csv('../raw_data/yellow_tripdata_2020-04.csv', encoding='utf8', engine='python')

In [7]:
prep_jan = preprocess(jan_yellow)
prep_feb = preprocess(feb_yellow)
prep_mar = preprocess(mar_yellow)
prep_apr = preprocess(apr_yellow)

In [16]:
# Export clean dfs to feather, ready for EDA in next notebook

prep_jan.reset_index().to_feather('clean_jan.feather')
prep_feb.reset_index().to_feather('clean_feb.feather')
prep_mar.reset_index().to_feather('clean_mar.feather')
prep_apr.reset_index().to_feather('clean_apr.feather')

In [8]:
prep_jan.shape

(6142038, 11)

# Dropping Columns
 
 Full drop VendorID, passenger_count, store_and_fwd_flag, extra, mta_tax, tolls_amount, improvement_surcharge, congestion_surcharge
 
 
 Partial drop | RatecodeID = 99 | payment_type = 5 | 

## Justification:

*** These details will be included in the report as well, but writing out here just in case

VendorID - Fully irrelevant; contains info about who provided data for the record

store_and_fwd_flag - Indicates whether or not the driver had a connection to the server at the time of the trip. Interesting to note that for 'N' values, drivers could have possible artificially inflated certain metrics. However, trips with an 'N' value are relatively scarce - only 10% for January, and so don't need to be considered

extra - Miscellaneous fees / charges. Currently only includes 2 possible charges, dependent on time of trip. These times are static, and specific rush hour times are provided on the TLC website. This analysis will not consider extra charges, nor their effect, and will concentrate more on the full fare amount

mta_tax - See above

tolls_amount - See above

improvement_surcharge - See above

RateCodeID - An ID of 99 corresponds to something outside of the scope of the dataset - No explanation for the 99 value was provided in the data dictionary

## Dropping rows based on column value:

payment_type - A value of 5 indicates an 'unknown' method of payment. This doesn't come up often; there was only 1 of these in the month of Jan

## Dropping NAN values:
Some rows contained nan values, and these were common cells between RatecodeID and payment_type. There was a tip amount attached to those instances, so the error isn't explained by a mixup with credit details. Either way, these nan rows made up less than 3% of the dataset, so they were dropped for simplicity

In [ ]:
jan_yellow['store_and_fwd_flag'].value_counts()

In [ ]:
jan_yellow['RatecodeID'].value_counts()

In [ ]:
jan_yellow['payment_type'].value_counts()

In [ ]:
full_drop_labels = ['VendorID', 'passenger_count', 'store_and_fwd_flag', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge']

revised = jan_yellow.drop(full_drop_labels, axis = 1)

print('# of columns dropped:', jan_yellow.shape[1] - revised.shape[1])

In [ ]:
drop_index = revised[(revised['payment_type'] == 5.0)].index
revised2 = revised.drop(drop_index)

print('Number of rows dropped:', revised.shape[0] - revised2.shape[0])

In [ ]:
revised2['RatecodeID'].value_counts()

In [ ]:
%time
revised3 = revised2.loc[revised2['RatecodeID'] != 99]

In [ ]:
revised3['tpep_pickup_datetime'] = pd.to_datetime(revised3['tpep_pickup_datetime'])
revised3['tpep_dropoff_datetime'] = pd.to_datetime(revised3['tpep_dropoff_datetime'])

In [1]:
revised3.dtypes

NameError: name 'revised3' is not defined

In [ ]:
revised3.columns

# Outlier Justification -

In [ ]:
display(prep_jan.describe())
print(prep_jan.shape)

# We can see that there are negative values in fare amount, trip distance, tip amount and total amount. The attributes
# related to the fare paid are all dependent on each other, so can sort by total_amount. Negative trip distance is the
# second issue. There were 13 entries recorded with a negative value, so draw that down to user input error.

# Large values are also potential outliers - total_amount can get up to $4000 which is not realistic.
# It appears that a general rule for NYC taxis are that they limit their trips to $500 max fare and/or 100 miles.
# Can drop entries with a trip amount over $500.

# Basic filtering too - there are a number of entries with a passenger count of 0, which obviously doesn't make sense. Could be 
# due either to input error or taxi drivers putting in fake rides to meet some sort of quota

In [ ]:
prep_jan = prep_jan.loc[(prep_jan['total_amount'] > 0) & (prep_jan['trip_distance'] > 0) & (prep_jan['total_amount'] < 500) & (prep_jan['passenger_count'] > 0)]
display(prep_jan.describe())
print(prep_jan.shape)

# Still a healthy 6 mil rows after filtering